In [1]:
# https://stackoverflow.com/questions/43599018/is-there-a-way-to-get-matplotlib-path-contains-points-to-be-inclusive-of-boundar
#I do quite like this command in Jupiter notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
#It makes things wider and not waste the space on your screen
import pandas as pd
import importlib
import csv,os
import numpy as np
import pandas as pd
import itertools
import pyproj
import shapely
import copy
from uafgi import gicollections,cfutil,glacier,gdalutil,shputil,ncutil
import uafgi.data.ns642
import netCDF4
import importlib
import subprocess

In [47]:
#grid = 'W71.65N'   # Rink Isbrae
grid = 'W70.90N'   # Perlerfiup Se.
itslive_nc = 'outputs/itslive/GRE_G0240_{}_2011_2018.nc'.format(grid)
bm_nc = 'outputs/bedmachine/BedMachineGreenland-2017-09-20_{}.nc'.format(grid)

In [49]:
importlib.reload(glacier)
with netCDF4.Dataset(bm_nc) as nc:
    bed = nc.variables['bed'][:]
    thk = nc.variables['thickness'][:]

thk = thk.astype('d') * .1
grid_info = gdalutil.FileInfo(itslive_nc)
print(grid_info.dx)
print(grid_info.dy)
    
with netCDF4.Dataset(itslive_nc) as nc:
    with netCDF4.Dataset('x2.nc', 'w') as ncout:
        nccopy = ncutil.copy_nc(nc,ncout)
        nccopy.copyDimensions(*nc.dimensions.keys())

        vnames = ('polar_stereographic', 'x', 'y')
        nccopy.define_vars(vnames, zlib=True)

        ncfvel = ncout.createVariable('fvel', 'd', ('time','y','x'), zlib=True)
        ncsigma = ncout.createVariable('sigma', 'd', ('time','y','x'), zlib=True)
        ncbed = ncout.createVariable('bed', 'd', ('y','x'), zlib=True)
        ncthk = ncout.createVariable('thickness', 'd', ('y','x'), zlib=True)

        for vname in vnames:
            nccopy.copy_var(vname)
        ncthk[:] = thk
        ncbed[:] = bed

        ntime = len(nc.dimensions['time'])
        unitconv = 1. / (365. * 86400.)
        for itime in range(0,ntime):
            u_surface = nc.variables['u_ssa_bc'][itime,:]
            u_surface *= unitconv
            v_surface = nc.variables['v_ssa_bc'][itime,:]
            v_surface *= unitconv
            fvel = thk * np.sqrt(u_surface*u_surface + v_surface*v_surface)
            sigma = glacier.von_mises_stress(u_surface, v_surface, thk, grid_info.dx, grid_info.dy)
            ncfvel[itime,:] = fvel
            ncsigma[itime,:] = sigma


100.0
100.0
Copying polar_stereographic
Copying x
Copying y


In [4]:
importlib.reload(glacier)
sigma = glacier.von_mises_stress(u_surface, v_surface)

masked_array(
  data=[[[ 6.75319366e+01,  6.83280869e+01,  6.91242371e+01, ...,
          -1.47815061e+00, -1.37317848e+00, -1.26152349e+00],
         [ 6.72331696e+01,  6.79185028e+01,  6.86038361e+01, ...,
          -1.40416253e+00, -1.28708839e+00, -1.16126728e+00],
         [ 6.69344025e+01,  6.75089188e+01,  6.80834351e+01, ...,
          -1.33017457e+00, -1.20099831e+00, -1.06101120e+00],
         ...,
         [ 4.41474304e+01,  4.40593529e+01,  4.39712791e+01, ...,
          -3.03138673e-01,  1.93710494e+00,  2.51135206e+00],
         [ 4.43941574e+01,  4.42938042e+01,  4.41934509e+01, ...,
          -5.07736349e+00, -1.19396591e+00, -1.43847436e-01],
         [ 4.45667191e+01,  4.44541931e+01,  4.43416672e+01, ...,
          -2.50920153e+00, -1.43847436e-01,  3.81211817e-01]],

        [[ 6.18530312e+01,  6.20281906e+01,  6.22033463e+01, ...,
          -1.31631970e+00, -1.21274412e+00, -1.12567997e+00],
         [ 6.23105316e+01,  6.25106506e+01,  6.27107773e+01, ...,
        

In [8]:
select = pd.read_pickle(uafgi.data.join_outputs('stability', '03_select.df'))
select

,w21_popular_name,w21_greenlandic_name,w21_coast,w21_category,w21_Qr,w21_Qf,w21_Qm,w21_Qs,w21_Qc_inferred,w21_qm,...,up_id,up_loc,w21t_key,w21t_Glacier,grid_info,fjord_classes,terminus,ns642_years,ns642_termini,retreat_rate
0,F. Graae Gl.,,CE,DW,1.67,4.03,3.58,0.00,2.12,0.02,...,None,POINT (546206.5823260495 -1872217.376889129),F Graae,F Graae,<uafgi.gdalutil.FileInfo object at 0x1141a7890>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (549489.6241700107 -1874776.7249394...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (549765.6446273515 -1874639.261997...,-62.792085
1,Perlerfiup Se.,Perlerfiup Sermia,CW,DW,3.38,0.91,1.53,0.00,2.76,0.46,...,None,POINT (-212388.8772227576 -2067498.956061578),Perlerfiup,Perlerfiup,<uafgi.gdalutil.FileInfo object at 0x1141d3350>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-213822.3790058642 -2068385.804775...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-214983.8014421279 -2064816.58070...,-244.187242
2,Sermeq Avannarleq,Sermeq Avannarleq,CW,NC,3.32,1.95,NaN,0.00,NaN,NaN,...,None,POINT (-194476.90501221 -2155226.816762347),Sermeq Avannarleq,Sermeq Avannarleq,<uafgi.gdalutil.FileInfo object at 0x1141d3390>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-204989.7710313856 -2172104.812982...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-205164.0096814953 -2172531.71591...,-15.842697
3,Kangilleq,Kangilleq,CW,SC,0.13,0.01,0.83,0.00,NaN,0.70,...,None,POINT (-201997.6296952416 -2091453.856903825),Kangilleq,Kangilleq,<uafgi.gdalutil.FileInfo object at 0x1141d33d0>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-208030.3059775533 -2096715.578089...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-208089.7902075725 -2096724.67727...,10.350601
4,Farquhar Gl.,Tuttulipaluup Sermia,N,SC,3.33,1.79,1.53,0.00,NaN,0.12,...,None,POINT (-481448.0888079512 -1242228.350246575),Farquhar,Farquhar,<uafgi.gdalutil.FileInfo object at 0x1141d3410>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-485919.2819863445 -1246152.210928...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-488080.3077002923 -1247612.29671...,-136.678031
5,Ussing Br.,Sermeq Kujalleq,NW,CR,0.48,0.39,3.64,0.00,NaN,NaN,...,None,POINT (-312776.5154749735 -1730232.681903133),Ussing Braeer,Ussing Braeer,<uafgi.gdalutil.FileInfo object at 0x1141d3450>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-323158.9168130422 -1729987.165057...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-322912.1385858558 -1729821.53117...,-26.064922
6,Kong Oscar Gl.,Nuussuup Sermia,NW,DW,4.17,0.36,4.46,1.11,-1.04,0.72,...,None,POINT (-351834.1906168468 -1448999.131146864),Kong Oscar,Kong Oscar,<uafgi.gdalutil.FileInfo object at 0x1141d3490>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-391116.5677449044 -1473344.663588...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-391966.9534006705 -1474673.54763...,-113.015465
7,Morell Gl.,,NW,NC,4.59,6.15,NaN,77.92,NaN,NaN,...,None,POINT (-448469.3585649088 -1421601.716236305),Morell,Morell,<uafgi.gdalutil.FileInfo object at 0x1141d34d0>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-452011.3745308681 -1422922.488703...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-452092.2692915126 -1423255.16822...,93.564640
8,Nordenskiold Gl. N,,NW,SC,0.61,0.97,0.36,0.00,NaN,0.35,...,None,POINT (-369289.6567919262 -1487510.253395633),Nordenskiold,Nordenskiold,<uafgi.gdalutil.FileInfo object at 0x1141d3510>,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",LINESTRING (-375643.796443776 -1493008.6474716...,"[2000, 2005, 2006, 2007, 2008, 2012, 2014, 201...",[LINESTRING (-375706.9534006705 -1492893.54763...,-776.694019
9,Rimfaxe Gl.,,SE,CR,0.09,-5.83,1.16,0.00,NaN,NaN,...,None,POINT (132178.8373676752 -2947988.219307346),Rimfaxe,Rimfaxe,<uafgi.gdalutil.FileInfo object at

In [9]:
select.columns

Index(['w21_popular_name', 'w21_greenlandic_name', 'w21_coast', 'w21_category',
       'w21_Qr', 'w21_Qf', 'w21_Qm', 'w21_Qs', 'w21_Qc_inferred', 'w21_qm',
       'w21_qf', 'w21_qc', 'w21_mean_depth', 'w21_min_depth',
       'w21_quality_str', 'w21_area_grounded_1992_2017',
       'w21_area_grounded_1992_1997', 'w21_area_grounded_1998_2007',
       'w21_area_grounded_2008_2017', 'w21_mean_fjord_width',
       'w21_length_grounded_1992_2017', 'w21_length_grounded_1992_1997',
       'w21_length_grounded_1998_2007', 'w21_length_grounded_2008_2017',
       'w21_ocean_model_sample_area', 'w21_mean_TF_1992-2017',
       'w21_mean_TF_1992-1997', 'w21_mean_TF_1998-2007',
       'w21_mean_TF_2008-2017', 'w21_subglacial_discharge_1992_2017',
       'w21_subglacial_discharge_1992_1997',
       'w21_subglacial_discharge_1998_2007',
       'w21_subglacial_discharge_2008_2017', 'w21_mean_xsection_area',
       'w21_mean_undercutting_1992_2017', 'w21_mean_undercutting_1992_1997',
       'w21_mean_und

In [12]:
import uafgi.data.w21 as d_w21
import uafgi.data.wkt
w21t = d_w21.read_termini(uafgi.data.wkt.nsidc_ps_north).df
w21t


,w21t_Glacier,w21t_Year,w21t_Day_of_Yea,w21t_Month,w21t_Day,w21t_terminus,w21t_key
0,Inngia,2016,193,7,11,"LINESTRING (-259155 -1941830, -259147 -1941820...","(Inngia, 2016, 193)"
1,Inngia,1985,132,5,12,"LINESTRING (-263302 -1948350, -263302 -1948280...","(Inngia, 1985, 132)"
2,Inngia,1999,298,10,25,"LINESTRING (-264009 -1945300, -264009 -1945310...","(Inngia, 1999, 298)"
3,Inngia,2018,200,7,19,"LINESTRING (-255747 -1942400, -255761 -1942380...","(Inngia, 2018, 200)"
4,Inngia,2001,191,7,10,"LINESTRING (-264031 -1945260, -264031 -1945280...","(Inngia, 2001, 191)"
...,...,...,...,...,...,...,...
6445,Upernavik Isstrom SS,2006,219,8,7,LINESTRING (-302638.850168814 -1853738.9960646...,"(Upernavik Isstrom SS, 2006, 219)"
6446,Upernavik Isstrom SS,1985,98,4,8,LINESTRING (-307346.194460371 -1851815.6366210...,"(Upernavik Isstrom SS, 1985, 98)"
6447,Upernavik Isstrom SS,2015,213,8,1,LINESTRING (-302071.267970121 -1854304.5691209...,"(Upernavik Isstrom SS, 2015, 213)"
6448,Upernavik Isstrom SS,1995,117,4,27,"LINESTRING (-305045 -1853340, -305018 -1853330...","(Upernavik Isstrom SS, 1995, 117)"


In [14]:
row = select.iloc[0].to_dict()

In [17]:
w21t[w21t['w21t_Glacier'] == row['w21t_Glacier']].sort_values(['w21t_Year', 'w21t_Day_of_Yea'])

,w21t_Glacier,w21t_Year,w21t_Day_of_Yea,w21t_Month,w21t_Day,w21t_terminus,w21t_key
3656,F Graae,1984,128,5,7,"LINESTRING (549523 -1878280, 549541 -1878260, ...","(F Graae, 1984, 128)"
3671,F Graae,1985,162,6,11,"LINESTRING (549087 -1877660, 549081 -1877610, ...","(F Graae, 1985, 162)"
3670,F Graae,1985,258,9,15,"LINESTRING (548996 -1877490, 549014 -1877440, ...","(F Graae, 1985, 258)"
3675,F Graae,1986,101,4,11,"LINESTRING (549051 -1877590, 549069 -1877500, ...","(F Graae, 1986, 101)"
3652,F Graae,1986,277,10,4,"LINESTRING (548984 -1877500, 549002 -1877440, ...","(F Graae, 1986, 277)"
3654,F Graae,1987,104,4,14,"LINESTRING (549044 -1877540, 549057 -1877490, ...","(F Graae, 1987, 104)"
3674,F Graae,1987,264,9,21,"LINESTRING (548709 -1877210, 548801 -1877230, ...","(F Graae, 1987, 264)"
3662,F Graae,1988,107,4,16,"LINESTRING (549044 -1877490, 549069 -1877460, ...","(F Graae, 1988, 107)"
3663,F Graae,1988,251,9,7,"LINESTRING (548789 -1877300, 548795 -1877290, ...","(F Graae, 1988, 251)"
3677,F Graae,1989,93,4,3,"LINESTRING (549044 -1877530, 549081 -1877490, ...","(F Graae, 1989, 93)"


In [18]:
w21t.columns

Index(['w21t_Glacier', 'w21t_Year', 'w21t_Day_of_Yea', 'w21t_Month',
       'w21t_Day', 'w21t_terminus', 'w21t_key'],
      dtype='object')